# **Classification of the ModelNet dataset with ORION**


**Installation of the libraries**

In [ ]:
#connect to Google drive to access datasets
from google.colab import drive
drive.mount('/content/drive')

#import libraries
import numpy as np
import torch
import h5py
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch.nn import CrossEntropyLoss


!pip install torchvision
from torchvision.transforms import Compose

#install wandb in order to obtain graphs
!pip install wandb
import wandb

# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



#**Data pre-processing**
**Functions**

In [ ]:
class ImportDatas(Dataset):
  def __init__(self, file_txt_path, transform=None):

        with open(file_txt_path) as file:
            lines = file.readlines()

        self.paths = [r"/content/drive/MyDrive/ORION_dataset/ModelNet10_24rot/" + line.strip('\n')[40:] for line in lines]
        self.data, self.label, self.label_pose = self._combine_files()
        self.transform = transform

  def _combine_files(self):
        combined_data = None
        combined_label = None
        combined_label_pose = None

        for path in self.paths:
            file = h5py.File(path, 'r')
            data = file['data']

            label = np.int64(file['label'])
            label_pose = np.int64(file['label_pose'])

            if combined_data is None:
                combined_data = data
                combined_label = label
                combined_label_pose = label_pose
            else:
                combined_data = np.concatenate((combined_data, data), axis=0)
                combined_label = np.concatenate((combined_label, label), axis=0)
                combined_label_pose = np.concatenate((combined_label_pose, label_pose), axis=0)

        return combined_data, combined_label, combined_label_pose

  def __len__(self):
        return len(self.data)

  def __getitem__(self, idx):
        data = self.data[idx]
        label = self.label[idx]
        label_pose = self.label_pose[idx]
        if self.transform:
            data = self.transform(data)
        return data, label, label_pose


In [ ]:
class CropTransform(torch.nn.Module):
  def forward(self, datas):
    #cropping datas from [36x36x36] to [32x32x32]
    cropped_datas = datas[: ,2:34, 2:34, 2:34]
    return cropped_datas

class ToTensor(torch.nn.Module):
  def forward(self, datas):
    tensor = torch.from_numpy(datas.astype(np.float32)) #transform datas into tensor
    return tensor

transforms = Compose([
    CropTransform(),
    ToTensor()
])

**Datas uploading from Google Drive**

In [ ]:
#define the type of set poseplan
num_classes=10
num_poses=210


# initialize paths
train_dataset = ImportDatas(r'/content/drive/MyDrive/ORION_dataset/ModelNet10_24rot/poseplan_MN10_24/hdf5/train_allrot/train.hdf5.txt', transform=transforms)
validation_dataset = ImportDatas(r'/content/drive/MyDrive/ORION_dataset/ModelNet10_24rot/poseplan_MN10_24/hdf5/validation_allrot/train.hdf5.txt', transform=transforms)
test_dataset = ImportDatas(r'/content/drive/MyDrive/ORION_dataset/ModelNet10_24rot/poseplan_MN10_24/hdf5/test_allrot/train.hdf5.txt', transform=transforms)

# load datas
#change batch size for validation and test. 12 if dealing with 12 rotations, 24 for 24 rotations
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=24, shuffle=False, num_workers=8, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=24, shuffle=False, num_workers=8)

#**Print one object with label**


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
iterator = tqdm(validation_dataloader, disable=True)
torch.set_printoptions(threshold=327680)
for datas, label_class, label_pose in iterator:
        datas = datas.squeeze().to(device)
        label_class = label_class.squeeze().to(device)
        label_pose = label_pose.squeeze().to(device)
        print("Batch Data Shape:", datas)
        #print("Batch Class Labels Shape:", label_class.shape)
        #print("Batch Pose Labels Shape:", label_pose.shape)

        print(label_class)
        print(label_pose)

# Add this after initializing Data Logging
#print("Length of Train Dataset:", len(train_dataset))
print("Length of Validation Dataset:", len(validation_dataset))
#print("Length of Test Dataset:", len(test_dataset))

# **ORION nets**
**Definition of the ORION net with the same architecture provided into the paper**

In [ ]:
from torch.nn import Module, Sequential, Conv3d, BatchNorm3d, ReLU, Dropout3d, MaxPool3d, Linear, LeakyReLU, Dropout, ELU

# Define a simple CNN architecture
class ORION(Module):
    def __init__(self, num_classes, num_poses):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=5, stride=2),
          BatchNorm3d(num_features=32),
          ReLU(),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          ReLU(),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.3),
          )

        self.fc1 = Sequential(
            Linear(64*6*6*6, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.pose_layer = Linear(128, num_poses)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        pose_output = self.pose_layer(x)
        return class_output, pose_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)
        torch.nn.init.normal_(self.pose_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

        if self.pose_layer.bias is not None:
            self.pose_layer.bias.data.zero_()

**Modified versions of the ORION net**

**Extended ORION (from Paper)**

In [ ]:
# Define a simple CNN architecture
class EXTENDED_ORION(Module):
    def __init__(self, num_classes, num_poses):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=3, stride=2),
          BatchNorm3d(num_features=32),
          ReLU(),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          ReLU(),
          Dropout3d(p=0.3),

          # Definition of Conv3
          Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1),
          BatchNorm3d(num_features=128),
          ReLU(),
          Dropout3d(p=0.4),

          # Definition of Conv4
          Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1),
          BatchNorm3d(num_features=256),
          ReLU(),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.6),
          )

        self.fc1 = Sequential(
            Linear(256*4*4*4, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.pose_layer = Linear(128, num_poses)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        pose_output = self.pose_layer(x)
        return class_output, pose_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)
        torch.nn.init.normal_(self.pose_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

        if self.pose_layer.bias is not None:
            self.pose_layer.bias.data.zero_()

**Using Leaky Relu as activation function**

In [ ]:
# Define a simple CNN architecture
class ORION_LRELU(Module):
    def __init__(self, num_classes, num_poses):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=5, stride=2),
          BatchNorm3d(num_features=32),
          LeakyReLU(negative_slope=0.1),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          LeakyReLU(negative_slope=0.1),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.3),
          )

        self.fc1 = Sequential(
            Linear(64*6*6*6, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.pose_layer = Linear(128, num_poses)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        pose_output = self.pose_layer(x)
        return class_output, pose_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)
        torch.nn.init.normal_(self.pose_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

        if self.pose_layer.bias is not None:
            self.pose_layer.bias.data.zero_()

In [ ]:
# Define a simple CNN architecture
class EXTENDED_ORION_LRELU(Module):
    def __init__(self, num_classes, num_poses):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=3, stride=2),
          BatchNorm3d(num_features=32),
          LeakyReLU(negative_slope=0.1),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          LeakyReLU(negative_slope=0.1),
          Dropout3d(p=0.3),

          # Definition of Conv3
          Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1),
          BatchNorm3d(num_features=128),
          LeakyReLU(negative_slope=0.1),
          Dropout3d(p=0.4),

          # Definition of Conv4
          Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1),
          BatchNorm3d(num_features=256),
          LeakyReLU(negative_slope=0.1),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.6),
          )

        self.fc1 = Sequential(
            Linear(256*4*4*4, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.pose_layer = Linear(128, num_poses)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        pose_output = self.pose_layer(x)
        return class_output, pose_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)
        torch.nn.init.normal_(self.pose_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

        if self.pose_layer.bias is not None:
            self.pose_layer.bias.data.zero_()


**Using Exponential- linear unit as activation function**

In [ ]:
# Define a simple CNN architecture
class ORION_ELU(Module):
    def __init__(self, num_classes, num_poses):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=5, stride=2),
          BatchNorm3d(num_features=32),
          ELU(),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          ELU(),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.3),
          )

        self.fc1 = Sequential(
            Linear(64*6*6*6, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.pose_layer = Linear(128, num_poses)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        pose_output = self.pose_layer(x)
        return class_output, pose_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)
        torch.nn.init.normal_(self.pose_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

        if self.pose_layer.bias is not None:
            self.pose_layer.bias.data.zero_()

In [ ]:
# Define a simple CNN architecture
class EXTENDED_ORION_ELU(Module):
    def __init__(self, num_classes, num_poses):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=3, stride=2),
          BatchNorm3d(num_features=32),
          ELU(),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          ELU(),
          Dropout3d(p=0.3),

          # Definition of Conv3
          Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1),
          BatchNorm3d(num_features=128),
          ELU(),
          Dropout3d(p=0.4),

          # Definition of Conv4
          Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1),
          BatchNorm3d(num_features=256),
          ELU(),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.6),
          )

        self.fc1 = Sequential(
            Linear(256*4*4*4, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.pose_layer = Linear(128, num_poses)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        pose_output = self.pose_layer(x)
        return class_output, pose_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)
        torch.nn.init.normal_(self.pose_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

        if self.pose_layer.bias is not None:
            self.pose_layer.bias.data.zero_()

**Only classes evaluation**

In [ ]:
# Define a simple CNN architecture
class ORION_OC(Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=5, stride=2),
          BatchNorm3d(num_features=32),
          ReLU(),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          ReLU(),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.3),
          )

        self.fc1 = Sequential(
            Linear(64*6*6*6, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        return class_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

In [ ]:
# Define a simple CNN architecture
class EXTENDED_ORION_OC(Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=3, stride=2),
          BatchNorm3d(num_features=32),
          ReLU(),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          ReLU(),
          Dropout3d(p=0.3),

          # Definition of Conv3
          Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1),
          BatchNorm3d(num_features=128),
          ReLU(),
          Dropout3d(p=0.4),

          # Definition of Conv4
          Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1),
          BatchNorm3d(num_features=256),
          ReLU(),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.6),
          )

        self.fc1 = Sequential(
            Linear(256*4*4*4, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        return class_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

#**Performances evaluation functions**
**Accuracies function**

In [ ]:
def accuracy (network, dataloader):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    network.to(device)
    softmax= torch.nn.Softmax(dim=1)

    iterator = tqdm(dataloader, disable=True)
    with torch.no_grad():
      predicted_class_array = []
      true_class_array = []
      predicted_pose_array = []
      true_pose_array = []

      # iterating through the batches
      for datas, label_class, label_pose in iterator:
        datas = datas.to(device)
        label_class = label_class.squeeze().to(device) #remove the dimesions of size 1
        label_pose = label_pose.squeeze().to(device)
        network.eval()
        # forward pass
        predicted_class, predicted_pose = network(datas)

        # obtain the class from the output
        sum_class_pred = torch.sum(predicted_class, dim=0)
        predicted_class = torch.argmax(sum_class_pred)
        # obtain the pose label from the output
        predicted_pose = torch.argmax(predicted_pose, dim=1)
        #update the arrays concatenating the results and the true labels
        predicted_class_array.append(predicted_class)
        true_class_array.append(label_class[0]) #only the first element is important, the others are just repetitions due to the fact that each batch is composed by rotations of the same object
        predicted_pose_array.append(predicted_pose)
        true_pose_array.append(label_pose) #in this case all the labels are important

      #build a single vector
      true_class_array= torch.stack(true_class_array)
      predicted_class_array= torch.stack(predicted_class_array)
      true_pose_array = torch.cat(true_pose_array, axis=0)
      predicted_pose_array= torch.cat(predicted_pose_array, dim=0)

      # calculate accuracy
      class_accuracy = torch.sum(predicted_class_array == true_class_array)/len(true_class_array)
      pose_accuracy = torch.sum(predicted_pose_array == true_pose_array)/len(true_pose_array)

      #return all the obtained elements
      return class_accuracy, pose_accuracy


**Losses function**

In [ ]:
def loss (network, dataloader):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    network.to(device)
    # defining the loss functions, creoss-entropy losses are chosen like in the paper
    loss_fn_class = CrossEntropyLoss()
    loss_fn_pose = CrossEntropyLoss()


    iterator = tqdm(dataloader, disable=True)

    with torch.no_grad():
      predicted_class_array = []
      true_class_array = []
      predicted_pose_array = []
      true_pose_array = []

      # iterating through the batches
      for datas, label_class, label_pose in iterator:
        datas = datas.to(device)
        label_class = label_class.squeeze().to(device) #remove the dimesions of size 1
        label_pose = label_pose.squeeze().to(device)
        network.eval()
        # forward pass
        predicted_class, predicted_pose = network(datas)

        predicted_class_array.append(predicted_class)
        true_class_array.append(label_class) #only the first element is import, the others are just repetions due to the fact that each batch is composed by rotations of the same object
        predicted_pose_array.append(predicted_pose)
        true_pose_array.append(label_pose) #in this case all the labels are important

      #build a single vector
      true_class_array= torch.cat(true_class_array, axis=0)
      predicted_class_array= torch.cat(predicted_class_array, axis=0)
      true_pose_array = torch.cat(true_pose_array, axis=0)
      predicted_pose_array = torch.cat(predicted_pose_array, axis=0)

      #calculate losses
      pose_loss = loss_fn_pose(predicted_pose_array, true_pose_array)
      class_loss = loss_fn_class(predicted_class_array, true_class_array)
      total_loss = (class_loss + pose_loss)/2 #weighted sum as defined in the paper

      #return all the obtained elements
      return class_loss, pose_loss, total_loss

**Accuracies function for class classificatio only**

In [ ]:
def accuracy_OC (network, dataloader):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    network.to(device)
    softmax= torch.nn.Softmax(dim=1)

    iterator = tqdm(dataloader, disable=True)
    with torch.no_grad():
      predicted_class_array = []
      true_class_array = []
      # iterating through the batches
      for datas, label_class, label_pose in iterator:
        datas = datas.to(device)
        label_class = label_class.squeeze().to(device) #remove the dimesions of size 1
        label_pose = label_pose.squeeze().to(device)
        network.eval()
        # forward pass
        predicted_class = network(datas)

        # obtain the class from the output
        sum_class_pred = torch.sum(predicted_class, dim=0)
        predicted_class = torch.argmax(sum_class_pred)
        #update the arrays concatenating the results and the true labels
        predicted_class_array.append(predicted_class)
        true_class_array.append(label_class[0]) #only the first element is important, the others are just repetitions due to the fact that each batch is composed by rotations of the same object

      #build a single vector
      true_class_array= torch.stack(true_class_array)
      predicted_class_array= torch.stack(predicted_class_array)

      # calculate accuracy
      class_accuracy = torch.sum(predicted_class_array == true_class_array)/len(true_class_array)

      #return all the obtained elements
      return class_accuracy


**Losses function for class classification only**

In [ ]:
def loss_OC (network, dataloader):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    network.to(device)
    # defining the loss functions, creoss-entropy losses are chosen like in the paper
    loss_fn_class = CrossEntropyLoss()

    iterator = tqdm(dataloader, disable=True)

    with torch.no_grad():
      predicted_class_array = []
      true_class_array = []

      # iterating through the batches
      for datas, label_class, label_pose in iterator:
        datas = datas.to(device)
        label_class = label_class.squeeze().to(device) #remove the dimesions of size 1
        label_pose = label_pose.squeeze().to(device)
        network.eval()
        # forward pass
        predicted_class = network(datas)

        predicted_class_array.append(predicted_class)
        true_class_array.append(label_class) #only the first element is import, the others are just repetions due to the fact that each batch is composed by rotations of the same object

      #build a single vector
      true_class_array= torch.cat(true_class_array, axis=0)
      predicted_class_array= torch.cat(predicted_class_array, axis=0)

      #calculate losses
      class_loss = loss_fn_class(predicted_class_array, true_class_array)

      #return all the obtained elements
      return class_loss

#**Data logging**
**Run this ti deal with class and orientation**

In [ ]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Modelnet10_multiple",

    # track hyperparameters and run metadata
    config={
    "dataset": "ModelNet10",
    "epochs": 200,
    }
)
wandb.define_metric("epoch")

wandb.define_metric("validation_loss", step_metric="epoch")
wandb.define_metric("class_validation_loss", step_metric="epoch")
wandb.define_metric("pose_validation_loss", step_metric="epoch")
wandb.define_metric("class_validation_accuracy", step_metric="epoch")
wandb.define_metric("pose_validation_accuracy", step_metric="epoch")
wandb.define_metric("class_validation_error", step_metric="epoch")
wandb.define_metric("pose_validation_error", step_metric="epoch")
#f4c123b14b87239cebd4b2c785e43afba6af6d07

**Run this to deal only with class**

In [ ]:
# start a new wandb
wandb.init(

    project="Modelnet10_onlyclassesSDG",

    config={
    "dataset": "ModelNet10",
    "epochs": 200,
    }
)
wandb.define_metric("epoch")
wandb.define_metric("class_validation_loss", step_metric="epoch")
wandb.define_metric("class_validation_accuracy", step_metric="epoch")
wandb.define_metric("class_validation_error", step_metric="epoch")

# **Training process**

In [ ]:
def training(model,train_data_loading, validation_data_loading, epochs, opt, early_stopping):

  device = "cuda" if torch.cuda.is_available() else "cpu"
  model.to(device)
  best_loss=np.inf #set the best value of the loss to infinite
  early_stopping_val=0
  class_loss_function = CrossEntropyLoss()
  pose_loss_function = CrossEntropyLoss()

  #start loop for each epoch, defined in input
  for epoch in range(epochs):

    print(f"Epoch: {epoch+1}")
    model.train()
    elements = tqdm(train_data_loading)
    for datas, label_class, label_pose in elements:
      #input real labels
      datas=datas.to(device)
      label_class=label_class.squeeze().to(device)
      label_pose=label_pose.squeeze().to(device)
      #do the forward pass
      label_class_prediction, label_pose_prediction=model(datas)

      #calculate losses
      class_loss = class_loss_function(label_class_prediction, label_class)
      pose_loss = pose_loss_function(label_pose_prediction, label_pose)
      train_loss = (class_loss + pose_loss)/2
      # logging training losses
      wandb.log({"train_loss": train_loss, "pose_train_loss": pose_loss, "class_train_loss": class_loss })

      #backword pass
      opt.zero_grad()
      train_loss.backward()
      opt.step()
      #view batch results
      elements.set_description(f"Train loss: {train_loss.detach().cpu().numpy()} class loss: {class_loss.detach().cpu().numpy()}  pose loss: {pose_loss.detach().cpu().numpy()}")
    #compute accuracies and losses on validation set
    class_accuracy_val, pose_accuracy_val = accuracy(model,validation_data_loading)
    class_loss_val, pose_loss_val, total_loss_val=loss(model, validation_data_loading)

    # logging validation results
    wandb.log({"epoch": epoch+1,
                "validation_loss": total_loss_val,
                "pose_validation_loss": pose_loss_val,
                "class_validation_loss": class_loss_val,
                "class_validation_accuracy": class_accuracy_val,
                "pose_validation_accuracy": pose_accuracy_val,
                "class_validation_error": 1-class_accuracy_val,
                "pose_validation_error": 1-pose_accuracy_val})

    #display elements
    print(f"class accuracy: {class_accuracy_val}, \n pose accuracy: {pose_accuracy_val}")
    print(f"class loss: {class_loss_val}, \n pose loss: {pose_loss} , \n total loss: {total_loss_val}")
    #updating model looking at the loss
    if total_loss_val < best_loss:
        print("Saved Model")
        #print(model)
        torch.save(model.state_dict(), "model.pt")
        best_loss = total_loss_val
        early_stopping_val=0 #re-initialize the count
    else:
        early_stopping_val+=1
        #check early stopping and exit training loop
    if early_stopping_val >= early_stopping :
      print("EARLY STOPPING")
      break

**Define a training process for the usage of calsses only, without considering the orientation**

In [ ]:
def training_OC(model,train_data_loading, validation_data_loading, epochs, opt, early_stopping):

  device = "cuda" if torch.cuda.is_available() else "cpu"
  model.to(device)
  best_loss=np.inf #set the best value of the loss to infinite
  early_stopping_val=0
  class_loss_function = CrossEntropyLoss()

  #start loop for each epoch, defined in input
  for epoch in range(epochs):

    print(f"Epoch: {epoch+1}")
    model.train()
    elements = tqdm(train_data_loading)
    for datas, label_class, label_pose in elements:
      #input real labels
      datas=datas.to(device)
      label_class=label_class.squeeze().to(device)
      label_pose=label_pose.squeeze().to(device)
      #do the forward pass
      label_class_prediction=model(datas)

      #calculate losses
      class_loss = class_loss_function(label_class_prediction, label_class)


      #backword pass
      opt.zero_grad()
      class_loss.backward()
      opt.step()
      #view batch results
      elements.set_description(f"class loss: {class_loss.detach().cpu().numpy()}")
    #compute accuracies and losses on validation set
    class_accuracy_val = accuracy_OC(model,validation_data_loading)
    class_loss_val = loss_OC(model, validation_data_loading)
    # logging validation results
    wandb.log({"epoch": epoch+1,
                "class_validation_loss": class_loss_val,
                "class_validation_accuracy": class_accuracy_val,
                "class_validation_error": 1-class_accuracy_val})

    #display elements
    print(f"class accuracy: {class_accuracy_val}")
    print(f"class loss: {class_loss_val}")
    #updating model looking at the loss
    if class_loss_val < best_loss:
        print("Saved Model")
        torch.save(model.state_dict(), "model.pt")
        best_loss = class_loss_val
        early_stopping_val=0 #re-initialize the count
    else:
        early_stopping_val+=1
        #check early stopping and exit training loop
    if early_stopping_val >= early_stopping :
      print("EARLY STOPPING")
      break

# **Initialization and running**

In [ ]:
from torch.optim import SGD, Adam

#manually define the architecture type
chosen_model=2;
#manually define the optimizaer type
chosen_opt=2;

#define epochs and early stopping
epochs=200
es=20

if chosen_model==1 and chosen_opt==1:
  model=ORION(num_classes, num_poses)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es) #91.5% sul test, 96% sul train
  #lr=0.001
  #momentum=0.9: pose acc 0.83(validation) 0.825(test)
  #momentum=0: pose acc 0.8044(validation) 0.8177(test)
  #momentum=0.4: pose acc 0.8011(validation) 0.8105(test)
elif chosen_model==1 and chosen_opt==2:
  model=ORION(num_classes, num_poses)
  opt = Adam(model.parameters(), lr=1e-4)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==2 and chosen_opt==1:
  model=EXTENDED_ORION(num_classes, num_poses)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True) #teoricamente va
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==2 and chosen_opt==2:
  model=EXTENDED_ORION(num_classes, num_poses)
  opt = Adam(model.parameters(), lr=1e-4) #teoricamente va
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)

#Leaky Relu
elif chosen_model==3 and chosen_opt==1:
  model=ORION_LRELU(num_classes, num_poses)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==3 and chosen_opt==2:
  model=ORION_LRELU(num_classes, num_poses)
  opt = Adam(model.parameters(), lr=1e-4)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==4 and chosen_opt==1:
  model=EXTENDED_ORION_LRELU(num_classes, num_poses)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==4 and chosen_opt==2:
  model=EXTENDED_ORION_LRELU(num_classes, num_poses)
  opt = Adam(model.parameters(), lr=1e-4)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)

#ELU
elif chosen_model==5 and chosen_opt==1:
  model=ORION_ELU(num_classes, num_poses)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==5 and chosen_opt==2:
  model=ORION_ELU(num_classes, num_poses)
  opt = Adam(model.parameters(), lr=1e-4)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==6 and chosen_opt==1:
  model=EXTENDED_ORION_ELU(num_classes, num_poses)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==6 and chosen_opt==2:
  model=EXTENDED_ORION_ELU(num_classes, num_poses)
  opt = Adam(model.parameters(), lr=1e-4)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)

#Only calsses evalyuation
elif chosen_model==7 and chosen_opt==1:
  model=ORION_OC(num_classes)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True) #validation accuracy 94.67% test 90.75%
  training_OC(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==7 and chosen_opt==2:
  model=ORION_OC(num_classes)
  opt = Adam(model.parameters(), lr=1e-4) #validation accuracy 93.33% test 90.86%
  training_OC(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==8 and chosen_opt==1:
  model=EXTENDED_ORION_OC(num_classes)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
  training_OC(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==8 and chosen_opt==2:
  model=EXTENDED_ORION_OC(num_classes)
  opt = Adam(model.parameters(), lr=1e-4)
  training_OC(model, train_dataloader, validation_dataloader, epochs, opt, es)



else:
  print("Indeces are wrong")


wandb.finish()

Epoch: 1


Train loss: 0.7462725639343262 class loss: 0.23589569330215454  pose loss: 1.2566494941711426: 100%|██████████| 2742/2742 [03:43<00:00, 12.25it/s]


class accuracy: 0.8833780288696289, 
 pose accuracy: 0.6513069868087769
class loss: 0.41891226172447205, 
 pose loss: 1.2566494941711426 , 
 total loss: 0.9178043007850647
Saved Model
Epoch: 2


Train loss: 0.7148725986480713 class loss: 0.27907687425613403  pose loss: 1.1506683826446533: 100%|██████████| 2742/2742 [03:47<00:00, 12.07it/s]


class accuracy: 0.9075067043304443, 
 pose accuracy: 0.7408958673477173
class loss: 0.3484748899936676, 
 pose loss: 1.1506683826446533 , 
 total loss: 0.6734451651573181
Saved Model
Epoch: 3


...

Train loss: 0.18905864655971527 class loss: 0.0980837419629097  pose loss: 0.28003355860710144: 100%|██████████| 2742/2742 [03:46<00:00, 12.08it/s]


class accuracy: 0.9142091274261475, 
 pose accuracy: 0.8239499926567078
class loss: 0.36500391364097595, 
 pose loss: 0.28003355860710144 , 
 total loss: 0.5272852778434753
Epoch: 31


Train loss: 0.08810223639011383 class loss: 0.02677772380411625  pose loss: 0.14942674338817596: 100%|██████████| 2742/2742 [03:47<00:00, 12.08it/s]


class accuracy: 0.9195711016654968, 
 pose accuracy: 0.8213807344436646
class loss: 0.36984366178512573, 
 pose loss: 0.14942674338817596 , 
 total loss: 0.5329952239990234
EARLY STOPPING


class_train_loss,▆█▆▃▆▃▆▂▂▄▄▅▂▁▂▂▁▂▃▂▃▁▂▂▂▂▂▁▁▂▁▂▁▃▁▁▁▂▃▁
class_validation_accuracy,▁▅▅▆▇▆██▅▅▆▅▇▆█▆▇▆▇▆▆▇▇▇▆▇█▇▆▆▇
class_validation_error,█▄▄▃▂▃▁▁▄▄▃▄▂▃▁▃▂▃▂▃▃▂▂▂▃▂▁▂▃▃▂
class_validation_loss,█▄▃▂▂▂▁▁▂▂▁▃▃▃▂▄▃▃▃▃▄▃▄▄▅▄▅▅▅▅▅
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
pose_train_loss,█▄▄▃▄▂▄▂▃▃▄▃▁▁▂▂▁▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂▁▂▂▁▂▁▂▂
pose_validation_accuracy,▁▅▆▆▇▇▇▇▇▇▇▇█▇█████████████████
pose_validation_error,█▄▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pose_validation_loss,█▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▅▅▃▄▂▄▂▃▃▄▃▁▁▂▂▁▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▁▃▂▁▂▁▂▂
validation_loss,█▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▂▂▂▂▂▂


# **Testing Phase**

In [ ]:
model.load_state_dict(torch.load("model.pt"))

val_class_accuracy, val_pose_accuracy = accuracy(model, validation_dataloader)
#val_class_accuracy = accuracy_OC(model, validation_dataloader)

print("Validation Class Accuracy : "+str(val_class_accuracy))
print("Validation Pose Accuracy : "+str(val_pose_accuracy))

test_class_accuracy, test_pose_accuracy = accuracy(model, test_dataloader)
#test_class_accuracy = accuracy_OC(model, test_dataloader)


print("Test Class Accuracy : "+str(test_class_accuracy))
print("Test Pose Accuracy : "+str(test_pose_accuracy))

Validation Class Accuracy : tensor(0.9155, device='cuda:0')
Validation Pose Accuracy : tensor(0.8107, device='cuda:0')
Test Class Accuracy : tensor(0.9335, device='cuda:0')
Test Pose Accuracy : tensor(0.8259, device='cuda:0')


# **Print mismatched objects**

In [ ]:
#!pip install h5py numpy matplotlib ipyvolume
#!pip install h5py matplotlib
import matplotlib.pyplot as plt
import ipyvolume as ipv
from tqdm import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
device = "cuda" if torch.cuda.is_available() else "cpu"

model=ORION(10,105)
model.to(device)


iterator = tqdm(test_dataloader, disable=True)
for datas, label_class, label_pose in iterator:
      datas = datas.to(device)
      label_class = label_class.squeeze().to(device) #remove the dimesions of size 1
      label_pose = label_pose.squeeze().to(device)

      predicted_class, predicted_pose=model(datas)
      sum_class_pred = torch.sum(predicted_class, dim=0)
      predicted_class = torch.argmax(sum_class_pred).item()

      for index in range(datas.size(0)):
        if predicted_class == label_class[index].item():
          print(datas.size())
          datass = datas[index].squeeze(0).cpu().detach().numpy()
          print("TRUE CLASS")
          print(label_class[index].item())


          # Create a 3D plot
          fig = plt.figure()
          ax = fig.add_subplot(111, projection='3d')

          # Create a meshgrid for the 3D data
          x, y, z = np.meshgrid(range(32), range(32), range(32))
          x, y, z = x.flatten(), y.flatten(), z.flatten()
          data_flat = datass.flatten()

          # Filter coordinates and data where the value is equal to 1
          x_filtered = z[data_flat == 1]
          y_filtered = x[data_flat == 1]
          z_filtered = y[data_flat == 1]

          # Plot the filtered points
          ax.scatter(x_filtered, y_filtered, z_filtered, c='blue', marker='o')

          # Customize the plot as needed
          ax.set_xlabel('X-axis')
          ax.set_ylabel('Y-axis')
          ax.set_zlabel('Z-axis')
          ax.set_title('3D Scatter Plot of HDF5 Data')

          # Show the plot
          plt.show()





**Show network**

In [ ]:
#!pip install torchkeras
#!pip install --upgrade torchkeras

#!pip install torchviz graphviz

import torch
from torch.autograd import Variable
from torchviz import make_dot

# Assuming ORION is your PyTorch model
model = ORION(num_classes, num_poses)

# Dummy input for visualization
dummy_input = Variable(torch.randn(1, 1, 32 , 32, 32))  # You may need to adjust the input size

# Generate a dynamic graph of the model
dot = make_dot(model(dummy_input), params=dict(model.named_parameters()))

# Display the graph directly in Colab
dot.render('/tmp/model_1', format='png', cleanup=True)
dot.render('/tmp/model_1', format='png', cleanup=True)

from IPython.display import Image
Image('/tmp/model_1.png')



In [ ]:
#!pip install torchsummary

from torchsummary import summary

# Create an instance of the model
model = EXTENDED_ORION(num_classes=10, num_poses=105)

# Print the summary to see the block scheme representation
summary(model, (1, 32, 32, 32), device="cpu")



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 32, 15, 15, 15]             896
       BatchNorm3d-2       [-1, 32, 15, 15, 15]              64
              ReLU-3       [-1, 32, 15, 15, 15]               0
         Dropout3d-4       [-1, 32, 15, 15, 15]               0
            Conv3d-5       [-1, 64, 13, 13, 13]          55,360
       BatchNorm3d-6       [-1, 64, 13, 13, 13]             128
              ReLU-7       [-1, 64, 13, 13, 13]               0
         Dropout3d-8       [-1, 64, 13, 13, 13]               0
            Conv3d-9      [-1, 128, 11, 11, 11]         221,312
      BatchNorm3d-10      [-1, 128, 11, 11, 11]             256
             ReLU-11      [-1, 128, 11, 11, 11]               0
        Dropout3d-12      [-1, 128, 11, 11, 11]               0
           Conv3d-13         [-1, 256, 9, 9, 9]         884,992
      BatchNorm3d-14         [-1, 256, 